In [1]:
import os
import numpy as np
import pandas as pd
from time import time

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 26

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [4]:
area = "nyc"
area_dir = os.path.join(data_dir, area)

reviews = pd.read_csv(os.path.join(area_dir, "nyc_reviews_nostopwords_en_20000samples.csv"))

print(reviews.shape)
print(reviews.columns)

(20000, 8)
Index(['Unnamed: 0', 'listing_id', 'id', 'date', 'reviewer_id',
       'reviewer_name', 'comments', 'normalized_comments'],
      dtype='object')


In [5]:
clean_reviews = reviews[pd.notnull(reviews.normalized_comments)]
clean_reviews.shape

(20000, 8)

Tokenize the documents

In [6]:
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
docs = [tokenizer.tokenize(review) for review in clean_reviews.normalized_comments]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

Create dictionary and bag-of-words corpus

In [9]:
from gensim import corpora, models
dictionary = corpora.Dictionary(docs)
print(len(dictionary))
#dictionary.filter_extremes(no_below=10, no_above=0.)
#print(len(dictionary))

14946


In [10]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Apply `gensim` LDA model

In [11]:
_ = dictionary[0]
id2word = dictionary.id2token

model = models.LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=3, passes=20)

In [12]:
model.print_topics()

[(0,
  '0.029*"not" + 0.015*"room" + 0.012*"apartment" + 0.010*"stay" + 0.010*"place" + 0.010*"us" + 0.009*"get" + 0.008*"bed" + 0.007*"night" + 0.007*"good"'),
 (1,
  '0.040*"stay" + 0.036*"great" + 0.029*"place" + 0.023*"host" + 0.019*"apartment" + 0.017*"location" + 0.015*"would" + 0.014*"clean" + 0.014*"recommend" + 0.011*"nice"'),
 (2,
  '0.023*"subway" + 0.021*"walk" + 0.020*"restaurant" + 0.016*"great" + 0.014*"manhattan" + 0.013*"close" + 0.012*"minute" + 0.010*"station" + 0.010*"apartment" + 0.009*"park"')]